In [1]:
import os
import csv
import math
import json
import numpy as np
import matplotlib.pyplot as plt

# Param Calculator

In [ ]:
# ANCHOR after running this run the next two cells
epsilon = 1.0;           
sigma = 1.0;                 
numParticles = 625;                       
temperature = 1.0;              
tau=math.sqrt(sigma*sigma/temperature)
gamma=100/tau
DR=0.08/tau
DT= temperature/gamma
Pe=(epsilon/temperature)-1
v=Pe * DT / sigma
# packingFraction = 0.0785
# boxSize = math.sqrt((numParticles*math.pi*(sigma/2)*(sigma/2))/packingFraction);
boxSize = 79;
packingFraction = (numParticles*math.pi*(sigma/2)*(sigma/2))/(boxSize**2);
#print all the variables
print ("epsilon = ", epsilon)
print ("sigma = ", sigma)
print ("numParticles = ", numParticles)
print ("boxSize = ", boxSize)
print ("temperature = ", temperature)
print ("gamma = ", gamma)
print ("tau = ", tau)
print ("DR = ", DR)
print ("DT = ", DT)
print ("Pe = ", Pe)
print ("v = ", v)
print ("packingFraction = ", packingFraction)
# 25 79 250

# Folder Renamer

In [ ]:
def get_new_folder_name(params):
    # Construct the new folder name based on the specified format
    new_name = f"B{int(float(params['boxSize']))}S{float(params['numSteps']) // 1000000}D{float(params['dataCompression']) // 10000}M{params['theta']}"
    return new_name

def rename_folders(parent_folder):
    subfolders = [f.path for f in os.scandir(parent_folder) if f.is_dir()]

    for folder in subfolders:
        params_file = os.path.join(folder, 'params.csv')

        if os.path.exists(params_file):
            with open(params_file, 'r') as csvfile:
                reader = csv.reader(csvfile)
                params = dict(reader)

                new_folder_name = get_new_folder_name(params)

                # Handle cases where the folder name already exists
                count = 1
                while os.path.exists(os.path.join(parent_folder, new_folder_name)):
                    new_folder_name = f"{get_new_folder_name(params)}({count})"
                    count += 1

                new_folder_path = os.path.join(parent_folder, new_folder_name)
            os.rename(folder, new_folder_path)
            print(f"Renamed {folder} to {new_folder_name}")


parent_folder_path = r"Runs\Low Density\DualRunPassive\D0.5Run"
rename_folders(parent_folder_path)

# JSON-DAT convertor

In [6]:
path=r"Wed_Jan_10_19_50_55_2024/"
def save_param(path):
    csv_file_path=path+"params.csv"
    # Open the CSV file
    with open(csv_file_path, mode='r') as infile:
        reader = csv.reader(infile)
        mydict = {rows[0]:rows[1] for rows in reader}
    return mydict
#takes in a json file as an numpy array
def load_data(path):
    with open(path+'particle_positions.json', 'r') as file:
        data = json.load(file)
    return data
variable_dict=save_param(path)
data = load_data(path)
particle_data = np.array(data)
positions = particle_data[:, :, :2]
boxSize=float(variable_dict["boxSize"])

In [8]:
# create a dat file for the positions by stacking the x and y coordinates for each particle in a row.
def create_dat_file(positions, box_size):
    for j in range(1, positions.shape[0]):
        d = positions[j, :] - positions[j-1, :]
        positions[j, :] -= np.round(d / box_size) * box_size
    # the first column is a serial number the second column is the x coordinate of the particles, the third column is the y coordinate of the particles
    positions = positions.reshape((positions.shape[0]*positions.shape[1],positions.shape[2]))
    # add the serial number as intger
    positions = np.column_stack((np.arange(1, positions.shape[0]+1, dtype=int), positions))
    np.savetxt(path + 'positions2.dat', positions, fmt=['%d', '%.6f', '%.6f'])
create_dat_file(positions,boxSize)